In [1]:
import pandas as pd
import copy
import random
from load_dataset import load
from classifier import NeuralNetwork, LogisticRegression, SVM
from utils import *
from metrics import *  # include fairness and corresponding derivatives
from expl import get_update_expl
from influence import *
from scipy.optimize import Bounds, minimize

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

In [2]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore') 

**Load Dataset**

In [3]:
dataset = 'german'
X_train, X_test, y_train, y_test = load(dataset)

In [4]:
X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Loss function** (Log loss for logistic regression)

In [5]:
# clf = NeuralNetwork(input_size=X_train.shape[-1])
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])
num_params = len(convert_grad_to_ndarray(list(clf.parameters())))
if isinstance(clf, LogisticRegression):
    loss_func = logistic_loss_torch
elif isinstance(clf, SVM):
    loss_func = svm_loss_torch
elif isinstance(clf, NeuralNetwork):
    loss_func = nn_loss_torch

In [6]:
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = NeuralNetwork(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])

clf.fit(X_train, y_train)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
print("Initial statistical parity: ", spd_0)

tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
print("Initial TPR parity: ", tpr_parity_0)

predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
print("Initial predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.09527580118738121
Initial TPR parity:  -0.07785149359511678
Initial predictive parity:  -0.10136869102808022
Initial loss:  0.5078892147492744
Initial accuracy:  0.755


In [7]:
metric = 0
if metric == 0:
    v1 = del_spd_del_theta(clf, X_test_orig, X_test, dataset)
elif metric == 1:
    v1 = del_tpr_parity_del_theta(clf, X_test_orig, X_test, y_test, dataset)
elif metric == 2:
    v1 = del_predictive_parity_del_theta(clf, X_test_orig, X_test, y_test, dataset)

**Update starts here**

In [8]:
# Learning Rate (can try with different step sizes and choose the best)
n = 50

In [9]:
# The pattern to be updated
idx = (X_train_orig.age==1)&(X_train_orig.gender==0)
delta_hist = []
final_out_hist = []
S = torch.Tensor(X_train[idx])
S.requires_grad = True
delta = torch.zeros(1, X_train.shape[-1])
delta.requires_grad = True
S_new = S + delta

In [10]:
part_1 = torch.FloatTensor(v1).repeat(len(S_new), 1).reshape(len(S_new), 1, -1)

In [11]:
part_2 = []
for i in range(len(S_new)):
    inner_lst = []
    del_L_del_theta_i_t = convert_grad_to_tensor(del_L_del_theta_i(clf, S_new[i], y_train[i],
                                                                   loss_func, retain_graph=True))
    for j in range(len(del_L_del_theta_i_t)):
        inner_grad = convert_grad_to_ndarray(grad(del_L_del_theta_i_t[j], delta, retain_graph=True))
        inner_lst.append(inner_grad)
    part_2.append(np.array(inner_lst))
    
part_2 = np.array(part_2)
part_2 = torch.FloatTensor(part_2)
part_2 = part_2.mean(dim=0).unsqueeze(0).repeat(len(S_new), 1, 1)
# part_2 = part_2.mean(dim=0).unsqueeze(0)

In [12]:
final = torch.bmm(part_1, part_2).reshape((len(S_new), -1))
delta = delta - n*final
S_new = S_new + delta

In [13]:
X_train_new = X_train.copy()
X_train_new[idx] = X_train_new[idx] + delta.detach().numpy()

**Project back to the sample space**

In [14]:
mins = []
maxs = []
numCols = len(X_train[0])
new_S = []
for i in range(numCols):
    mins.insert(i, min(X_train[:, i]))
    maxs.insert(i, max(X_train[:, i]))

bounds = Bounds(mins, maxs)
tbar = tqdm.tqdm(total=len(S_new))
for i in X_train_orig[idx].index:
    X_train_pert_pt = X_train_new[i]
    f = lambda x: np.linalg.norm(x - X_train_pert_pt)

    x0 = np.random.rand(numCols)
    res = minimize(f, x0, method='trust-constr', options={'verbose': 0}, bounds=bounds)
    X_train_new[i] = res.x
    tbar.update(1)

100%|██████████| 40/40 [00:10<00:00,  4.14it/s]

In [17]:
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = NeuralNetwork(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])

clf.fit(X_train_new, y_train)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

if metric==0:
    spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
    print("Updated statistical parity: ", spd_0)
elif metric==1:
    tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
    print("Updated TPR parity: ", tpr_parity_0)
else:
    predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
    print("Updated predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Updated loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Updated accuracy: ", accuracy_0)

Updated statistical parity:  -0.018946960102766752
Updated loss:  0.5159324960607171
Updated accuracy:  0.775
